In [1]:
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import pandas as pd
import requests
from bs4 import BeautifulSoup
from dotenv.main import load_dotenv

# .env 파일에서 환경변수 가져오기
load_dotenv()

# 공지사항 라우터 주소
notice_router = "https://www.anyang.ac.kr/main/communication/notice.do"

# 학사 공지사항 URL
school_url = 'https://www.anyang.ac.kr/main/communication/notice.do?mode=list&srSearchKey=&srSearchVal=&srCategoryId=18'

# AI융합학과 강의 정보가 담김 csv 경로
csv_path = "./AI융합학과_utf.csv"


# 도구 정의
@tool
def get_notices() -> str:
    """안양대학교 학사 공지사항 목록을 반환한다."""
    resp = requests.get(school_url)
    soup = BeautifulSoup(resp.content, 'html.parser')

    notice_divs = soup.find_all('div', {'class': "b-title-box"})

    notice_list = []
    for tr in notice_divs:
        link_tag = tr.find_all('a')[0]
        notice_list.append(f"\t{{\n\t\t제목: {link_tag.get('title')}, \n\t\t주소: {notice_router}{link_tag.get('href')}\n\t}}")

    return "[\n{0}\n]".format(",\n".join(notice_list))


@tool
def get_one_notice(notice_url: str) -> str:
    """특정 공지사항의 본문을 반환한다."""
    resp = requests.get(notice_url)
    soup = BeautifulSoup(resp.content, 'html.parser')
    notice_content = soup.find('div', {'class': 'b-main-box'})
    return notice_content.text.replace('\n\n', '').replace('\t', '')


@tool
def get_csv() -> str:
    """만양대학교 AI융합학과 강의 정보를 테이블 형태로 반환한다."""
    df = pd.read_csv(csv_path)
    return df.to_string()


# prompt 생성 (history 추가해야 함)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful anyang university assistant",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


# 이전에 정의한 도구 사용
tools = [get_notices, get_one_notice, get_csv]

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    # 추정 과정 출력
    verbose=True,
    handle_parsing_errors=True,
)

# 질문
query = "동계 계절수업 공지에 대해 알려줘"

# 실행
result = agent_executor.invoke({"input": query})

# 결과 출력
print("-----------------------------------------------------------"
      "실제 출력문:")
print(result["output"])




> Entering new AgentExecutor chain...

Invoking: `get_notices` with `{}`


[
	{
		제목: 범계역 청년출구 사업 관련 별정직 채용 공고 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34590&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: TEPS 시행으로 인한 수리관 일시사용중지 협조 요청[12/1] 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34582&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 2학기 학부 분납 4차 등록금 납부 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34541&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 2학기 대학원 분납 4차 등록금 납부 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34540&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 안양대학교 동계 계절수업 안내  자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34534&article.offset=0&articleLimit=15&srCate